In [1]:
import numpy as np
from numpy.linalg import norm

In [2]:
def initialization(n_atoms, T_init, dt, box):
    
    c_xyz = np.zeros((n_atoms,3))
    m_xyz = np.zeros((n_atoms,3))
    v_xyz = np.zeros((n_atoms,3))
    
    #c_dump = str(n_atoms) + '\ntitle\n'
    v_com = np.zeros((3)) 
    e_kin = 0
    for N in range(n_atoms):
        c_xyz[N] = np.random.rand(3)*box
        #c_dump += ' '.join(['O',str(c_xyz[N,0]),str(c_xyz[N,1]),str(c_xyz[N,2]),'\n'])
        v_xyz[N] = np.random.rand(3)-0.5
        v_com += v_xyz[N]
        e_kin += np.sum(v_xyz[N]**2)

    v_com /= n_atoms
    e_kin /= n_atoms
    scaling_fac = np.sqrt(3*T_init/e_kin)

    for N in range(n_atoms):
        v_xyz[N] -= v_com
        v_xyz[N] *= scaling_fac
        m_xyz[N] = c_xyz[N] - v_xyz[N]*dt
    
    c_dump = []
    return c_xyz, m_xyz, v_xyz, c_dump

In [3]:
def evaluate_force(n_atoms, c_xyz, box, LJ_cutoff):

    forces = np.zeros((n_atoms,3))
    for Ni in range(n_atoms-1):
        pos_i = np.array([c_xyz[Ni,0],c_xyz[Ni,1],c_xyz[Ni,2]])
        for Nj in np.arange(Ni+1,n_atoms):
            pos_j = np.array([c_xyz[Nj,0],c_xyz[Nj,1],c_xyz[Nj,2]])
            dij = np.remainder(pos_i - pos_j + box/2., box) - box/2.
            rij = norm(dij)
            if rij < LJ_cutoff:
                LJ_forces = 48*1/rij**2*1/rij**6*(1/rij**6-0.5)
                forces[Ni] += LJ_forces*dij
                forces[Nj] -= LJ_forces*dij
    return forces

In [29]:
def update_positions(n_atoms,c_xyz,m_xyz,forces):
    for Ni in range(n_atoms):
        n_xyz = 2*c_xyz[Ni]-m_xyz[Ni]+dt**2*forces[Ni]
        v_xyz = (n_xyz-m_xyz[Ni])/(2*dt)

        if boundary_conditions == 'periodic':
            if (n_xyz > box/2).any():
                n_xyz[n_xyz > box/2] -= box[n_xyz > box/2]
            if (n_xyz < -box/2).any():
                n_xyz[n_xyz < -box/2] += box[n_xyz < -box/2]

        m_xyz[Ni] = c_xyz[Ni]
        c_xyz[Ni] = n_xyz
        
    return c_xyz, m_xyz

In [56]:
def write_dump(t, c_dump, c_xyz, delta_dump, n_atoms):
    
    if t%10 == 0:
        if t == 0:
            c_dump = ''
        c_dump += str(len(c_xyz)) + '\ntitle\n'
        for i in range(len(c_xyz)):
            c_dump += ' '.join(['C',
                                str(c_xyz[i][0]),
                                str(c_xyz[i][1]),
                                str(c_xyz[i][2]),
                                '\n'])        
    return c_dump

In [57]:
n_atoms = 10
T_init = 1.0
dt = 0.005
box = np.array([20, 20, 20])
LJ_cutoff = 3
delta_dump = 10
boundary_conditions = 'periodic'

c_xyz, m_xyz, v_xyz, c_dump = initialization(n_atoms, T_init, dt, box)
for t in range(1000):
    forces = evaluate_force(n_atoms, c_xyz, box, LJ_cutoff)
    c_xyz, m_xyz = update_positions(n_atoms,c_xyz,m_xyz,forces)
    c_dump = write_dump(t, c_dump, c_xyz, delta_dump, n_atoms)

In [88]:
import py3Dmol
view = py3Dmol.view()
view.addModelsAsFrames(c_dump,'xyz')
view.animate({'loop': 'forward', 'reps': 1})
#view.animate({'loop': 'backAndForth'})
view.setStyle({'sphere':{'radius': 0.5}})
#view.setStyle({'stick':{}})
#view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
#view.addBox()
view.setBackgroundColor('black')
#view.setCameraParameters('orthographic')
view.setProjection("orthographic")
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':box[0]/2,'z':box[0]/2},'end':{'x':box[0]/2,'y':box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':-box[0]/2,'z':box[0]/2},'end':{'x':box[0]/2,'y':-box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':-box[0]/2,'z':box[0]/2},'end':{'x':-box[0]/2,'y':box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':box[0]/2,'y':-box[0]/2,'z':box[0]/2},'end':{'x':box[0]/2,'y':box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':box[0]/2,'z':-box[0]/2},'end':{'x':box[0]/2,'y':box[0]/2,'z':-box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':-box[0]/2,'z':-box[0]/2},'end':{'x':box[0]/2,'y':-box[0]/2,'z':-box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':-box[0]/2,'z':-box[0]/2},'end':{'x':-box[0]/2,'y':box[0]/2,'z':-box[0]/2}})
view.addLine({'color':'white','start':{'x':box[0]/2,'y':-box[0]/2,'z':-box[0]/2},'end':{'x':box[0]/2,'y':box[0]/2,'z':-box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':-box[0]/2,'z':-box[0]/2},'end':{'x':-box[0]/2,'y':-box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':box[0]/2,'y':-box[0]/2,'z':-box[0]/2},'end':{'x':box[0]/2,'y':-box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':-box[0]/2,'y':box[0]/2,'z':-box[0]/2},'end':{'x':-box[0]/2,'y':box[0]/2,'z':box[0]/2}})
view.addLine({'color':'white','start':{'x':box[0]/2,'y':box[0]/2,'z':-box[0]/2},'end':{'x':box[0]/2,'y':box[0]/2,'z':box[0]/2}})
#view.addUnitCell()
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [49]:
view.viewergrid

In [24]:
n_xyz

array([-5.9362624 , -0.13080443, -8.53494743])

In [26]:
c_xyz

array([[-5.3353779 ,  1.60122455,  2.3506134 ],
       [ 0.453994  , -9.24702121, -6.79752936],
       [ 3.93597057,  9.02523284, -8.22827058],
       [ 0.29816689,  2.55683094, -9.76387719],
       [ 0.83590054,  3.04248354,  3.15595424],
       [ 8.44925522, -7.8174413 , -0.218432  ],
       [-5.30474631, -9.37374504, -9.73210194],
       [-8.08707183,  9.40586843, -1.64869987],
       [ 4.91040375,  3.59120595,  0.08700255],
       [-5.9362624 , -0.13080443, -8.53494743]])

In [27]:
box/2

array([10., 10., 10.])